In [14]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import io
import pickle
import csv
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from datetime import datetime

In [2]:
os.chdir('..')

In [3]:
gauth = GoogleAuth()
gauth.DEFAULT_SETTINGS['client_config_file'] = 'client_secret_1057507276332-5mk9ac9q22rsmtm1idlqvpraq08ar8p5.apps.googleusercontent.com.json'
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

C:\Users\Jim\AppData\Roaming\Python\Python311\site-packages\oauth2client\_helpers.py:255: UserWarning: Cannot access mycreds.txt: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1057507276332-5mk9ac9q22rsmtm1idlqvpraq08ar8p5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [12]:
max_games = 500000 
asset_dir = 'asset'
file_name = '2023_tc_50000_games.pgn'

cached_urls_file = file_name.split('.')[0] + '_urls_list.pkl'
cached_ratings_file = file_name.split('.')[0] + '_ratings_list.pkl'
cached_games_file = file_name.split('.')[0] + '_game_arrays.pkl'

In [13]:
def find_folder_id(folder_name):
    """Find and return the Google Drive folder ID for a given folder name."""
    file_list = drive.ListFile({'q': f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
    for file in file_list:
        if file['title'] == folder_name:
            return file['id']
    return None

def read_pkl_file_from_drive(file_title, parent_id):
    """Read a .pkl file directly from Google Drive into a Python object."""
    query = f"'{parent_id}' in parents and trashed=false and title='{file_title}'"
    file_list = drive.ListFile({'q': query}).GetList()
    if not file_list:
        print(f"No file found with title: {file_title}")
        return None
    file = file_list[0]  # Assuming the first match is the correct one
    file_content = file.GetContentString(encoding='cp437')
    buffer = io.BytesIO(file_content.encode('cp437'))
    return pickle.load(buffer)

# Find the 'asset' folder ID
asset_folder_id = find_folder_id(asset_dir)
if asset_folder_id is None:
    print("Asset folder not found.")
else:
    # Adapt the variables for .pkl files to fetch them directly from Drive
    file_titles = {
        'urls_list': cached_urls_file,
        'ratings_list': cached_ratings_file,
        'game_arrays': cached_games_file,
    }

    # Load each .pkl file directly from Drive
    urls_list = read_pkl_file_from_drive(file_titles['urls_list'], asset_folder_id)
    ratings_list = read_pkl_file_from_drive(file_titles['ratings_list'], asset_folder_id)
    game_arrays = read_pkl_file_from_drive(file_titles['game_arrays'], asset_folder_id)

    # Example usage
    if urls_list is not None:
        print("URLs list loaded successfully.")
    if ratings_list is not None:
        print("Ratings list loaded successfully.")
    if game_arrays is not None:
        print("Game arrays loaded successfully.")

##2m 26.4s

URLs list loaded successfully.
Ratings list loaded successfully.
Game arrays loaded successfully.


In [15]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc_classification = nn.Linear(hidden_size, num_classes)
        self.fc_regression = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 

        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        
        out = self.dropout(F.relu(self.fc1(out)))
        out = self.dropout(F.relu(self.fc2(out)))
        out = self.dropout(F.relu(self.fc3(out)))
        out = self.dropout(F.relu(self.fc4(out)))
        out = self.dropout(F.relu(self.fc5(out)))
        out = self.dropout(F.relu(self.fc6(out)))
        out = self.dropout(F.relu(self.fc7(out)))
        classification_output = self.fc_classification(out)
        regression_output = self.fc_regression(out)
        return classification_output, regression_output

def combined_loss(classification_output, regression_output, target, alpha=0.5):
    classification_loss = nn.CrossEntropyLoss()(classification_output, target)
    regression_target = target.float()
    regression_loss = nn.MSELoss()(regression_output.squeeze(), regression_target)
    return alpha * classification_loss + (1 - alpha) * regression_loss

def train_model(model, train_loader, test_loader, optimizer, num_epochs, device, alpha=0.5):
    torets = []
    for epoch in range(num_epochs):
        model.train()
        for i, (moves, labels) in enumerate(train_loader):  
            moves = moves.to(device)
            labels = labels.to(device)

            classification_output, regression_output = model(moves)
            loss = combined_loss(classification_output, regression_output, labels, alpha)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        predicted_probs, predicted_labels, actual_labels = test_model(model, test_loader, device)
        pred_closeness = [sum(abs(p - a) <= k for p, a in zip(predicted_labels, actual_labels)) for k in range(10)]
        toret = [x/20000 for x in pred_closeness]
        torets.append(toret)
    return torets

def test_model(model, test_loader, device):
    model.eval()
    n_correct = 0
    n_samples = 0
    predicted_probs = []
    predicted_labels = []
    actual_labels = []
    with torch.no_grad():
        for moves, labels in test_loader:
            moves = moves.to(device)
            labels = labels.to(device)
            classification_output, _ = model(moves)
            probabilities = F.softmax(classification_output, dim=1)

            _, predicted = torch.max(classification_output.data, 1)
            predicted_probs.extend(probabilities.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the test moves: {acc} %')
    return predicted_probs, predicted_labels, actual_labels

# PIECE: [0, 1, 2, 4, 5, 9, 10, 11, 12, 14, 15, 27, 28, 29, 30, 31, 32]
# TIME: [6, 7, 8, 33, 34, 35, 36]
# ENGINE: [3, 37, 38, 39, 40, 41]
# DOMAIN: [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

def get_loaders(padded_games, ratings_list, urls_list, batch_size, fold_number=0):
    if fold_number < 0 or fold_number > 4:
        raise ValueError("fold_number must be between 0 and 4")
    test_list = padded_games[fold_number::5]
    #print(len(test_list))
    train_list = [df for i in range(5) if i != fold_number for df in padded_games[i::5]]
    test_ratings = ratings_list[fold_number::5]
    train_ratings = [ratings for i in range(5) if i != fold_number for ratings in ratings_list[i::5]]
    test_urls = urls_list[fold_number::5]
    train_urls = [url for i in range(5) if i != fold_number for url in urls_list[i::5]]

    train_data = [torch.FloatTensor(doc) for doc in train_list]
    test_data = [torch.FloatTensor(doc) for doc in test_list]
    train_labels = torch.LongTensor(train_ratings)
    test_labels = torch.LongTensor(test_ratings)

    train_dataset = TensorDataset(torch.stack(train_data), train_labels)
    test_dataset = TensorDataset(torch.stack(test_data), test_labels)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, train_urls, test_urls


def pad_game(game, max_length=256, vector_size=42):
    padding_length = max_length - len(game)
    if padding_length < 0:
        return game[:max_length]
    else:
        padding = np.full((padding_length, vector_size), -1)
        return np.vstack((game, padding))

In [16]:
# Hyperparameter ranges
input_size = 42
num_classes = 10
num_epochs = 8
sequence_lengths = [80] #1
batch_sizes = [100] #3
hidden_sizes = [128] #1
num_layers_list = [2] #1
learning_rates = [0.001] #1
alphas = [0.8] #4
dropout_rates = [0.25] #3
decays = [0.00001] #3
torch.manual_seed(64)

t = datetime.now().strftime("%Y-%m-%d-%H")
csv_file = "hyperparameter_tuning_results_{}.csv".format(t)
csv_columns = ['decay', 'dropout_rate', 'alpha_value', 'hidden_size', 'num_layers', 'learning_rate', 'batch_size', 'sequence_length', 'fold_number', 'lists']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
total_configs = len(sequence_lengths) * len(batch_sizes) * 5 * len(hidden_sizes) * len(num_layers_list) * len(learning_rates) * len(dropout_rates) * len(alphas) * len(decays)
current_config = 0

with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for decay in decays:
        for dropout_rate in dropout_rates:
            for alpha_value in alphas:
                for hidden_size in hidden_sizes:
                    for num_layers in num_layers_list:
                        for learning_rate in learning_rates:
                            for batch_size in batch_sizes:
                                for sequence_length in sequence_lengths:
                                    padded_games = [pad_game(g, sequence_length) for g in game_arrays]
                                    for fold_number in range(5):
                                        train_loader, test_loader, _, _ = get_loaders(padded_games, ratings_list, urls_list, batch_size, fold_number)

                                        # Increment the configuration counter
                                        current_config += 1
                                        print(f'Testing configuration {current_config} out of {total_configs}: Sequence Length={sequence_length}, Batch Size={batch_size}, Fold Number={fold_number}, Hidden Size={hidden_size}, Num Layers={num_layers}, Learning Rate={learning_rate}, Dropout Rate={dropout_rate}, Alpha={alpha_value}, Decay={decay}')
                                        
                                        model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
                                        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=decay)
                                        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                                        print(f'The model has {num_params:,} trainable parameters')

                                        lists = train_model(model, train_loader, test_loader, optimizer, num_epochs, device, alpha_value)

                                        row = {
                                            'decay': decay,
                                            'dropout_rate': dropout_rate,
                                            'alpha_value': alpha_value,
                                            'hidden_size': hidden_size,
                                            'num_layers': num_layers,
                                            'learning_rate': learning_rate,
                                            'batch_size': batch_size,
                                            'sequence_length': sequence_length,
                                            'fold_number': fold_number,
                                            'lists': lists
                                        }
                                        writer.writerow(row)

Testing configuration 1 out of 5: Sequence Length=80, Batch Size=100, Fold Number=0, Hidden Size=128, Num Layers=2, Learning Rate=0.001, Dropout Rate=0.25, Alpha=0.8, Decay=1e-05
The model has 337,163 trainable parameters
Epoch [1/32], Loss: 2.2423
Accuracy of the network on the test moves: 19.24 %
Epoch [2/32], Loss: 2.1171
Accuracy of the network on the test moves: 25.17 %
Epoch [3/32], Loss: 2.1309
Accuracy of the network on the test moves: 26.465 %
Epoch [4/32], Loss: 2.0242
Accuracy of the network on the test moves: 27.515 %
Epoch [5/32], Loss: 1.7415
Accuracy of the network on the test moves: 27.75 %
Epoch [6/32], Loss: 1.8806
Accuracy of the network on the test moves: 30.155 %
Epoch [7/32], Loss: 1.9554
Accuracy of the network on the test moves: 29.695 %
Epoch [8/32], Loss: 1.8391
Accuracy of the network on the test moves: 30.215 %
Epoch [9/32], Loss: 1.8034
Accuracy of the network on the test moves: 30.885 %
Epoch [10/32], Loss: 1.7174
Accuracy of the network on the test moves: